# Fator de Visão de Céu a partir do LiDAR com Ray Casting

Essa tentativa vai utililzar uma técnica análoga ao Ray Casting para tentar simplificar e melhorar a performance

In [36]:
import pdal
import numpy as np

In [37]:
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime


In [38]:
pipeline="""{
  "pipeline": [
    {
        "type": "readers.las",
        "filename": "arquivos/MDS/MDS_3314-231.laz"
    },
    {
        "type": "filters.sample",
        "radius": 1
    }
  ]
}"""

# pipeline="""{
#   "pipeline": [
#     {
#         "type": "readers.las",
#         "filename": "arquivos/MDS/MDS_3314-231.laz"
#     }
#   ]
# }"""

In [39]:
r = pdal.Pipeline(pipeline)
r.validate()
r.execute()
arrays = r.arrays

In [40]:
len(arrays[0])

471962

In [41]:
arrays[0][0]

(333499.29, 7395564.35, 735.34, 18, 1, 1, 0, 0, 6, -14., 17, 6, 356569.36606547)

In [42]:
r.schema

{'schema': {'dimensions': [{'name': 'X', 'size': 8, 'type': 'floating'},
   {'name': 'Y', 'size': 8, 'type': 'floating'},
   {'name': 'Z', 'size': 8, 'type': 'floating'},
   {'name': 'Intensity', 'size': 2, 'type': 'unsigned'},
   {'name': 'ReturnNumber', 'size': 1, 'type': 'unsigned'},
   {'name': 'NumberOfReturns', 'size': 1, 'type': 'unsigned'},
   {'name': 'ScanDirectionFlag', 'size': 1, 'type': 'unsigned'},
   {'name': 'EdgeOfFlightLine', 'size': 1, 'type': 'unsigned'},
   {'name': 'Classification', 'size': 1, 'type': 'unsigned'},
   {'name': 'ScanAngleRank', 'size': 4, 'type': 'floating'},
   {'name': 'UserData', 'size': 1, 'type': 'unsigned'},
   {'name': 'PointSourceId', 'size': 2, 'type': 'unsigned'},
   {'name': 'GpsTime', 'size': 8, 'type': 'floating'}]}}

## Importando dados para Pandas

In [43]:
import pandas as pd

In [53]:
df = pd.DataFrame(arrays[0])

In [54]:
df

,X,Y,Z,Intensity,ReturnNumber,NumberOfReturns,ScanDirectionFlag,EdgeOfFlightLine,Classification,ScanAngleRank,UserData,PointSourceId,GpsTime
0,333499.29,7395564.35,735.34,18,1,1,0,0,6,-14.0,17,6,356569.366065
1,333499.23,7395565.79,735.41,18,1,1,0,0,6,-14.0,17,6,356569.366090
2,333499.20,7395566.79,735.25,15,1,1,0,0,6,-14.0,17,6,356569.366105
3,333499.15,7395568.21,735.37,14,1,1,0,0,6,-14.0,17,6,356569.366129
4,333499.10,7395569.68,735.30,17,1,1,0,0,6,-15.0,17,6,356569.366153
...,...,...,...,...,...,...,...,...,...,...,...,...,...
471957,333523.36,7395570.78,735.52,2,1,2,1,0,19,13.0,40,6,363044.324524
471958,333521.03,7395570.91,731.07,5,1,1,1,0,6,13.0,40,6,363044.342732
471959,333520.96,7395571.00,732.50,8,1,1,1,0,6,13.0,40,6,363044.342740
471960,333521.09,7395571.84,731.83,18,1,1,1,0,6,13.0,40,6,363044.360909


In [55]:
df.Classification.unique()

array([ 6, 20,  2, 19,  5,  9], dtype=uint8)

In [56]:
df.Z.min() -  df.Z.max()

-187.37

In [57]:
df.Z = df.Z - df.Z.min() + ((df.Z.min() -  df.Z.max())/2)

In [59]:
df.Z.min()

-93.685

## Pre-Processando o Conjunto de Dados

In [60]:
pd.DataFrame.from_records(df[(df.Classification == 2) | (df.Classification == 6)]\
                          [['X', 'Y', 'Z']]).dtypes

X    float64
Y    float64
Z    float64
dtype: object

In [61]:
df_1m = \
df[(df.Classification == 2) | (df.Classification == 6)]\
                          [['X', 'Y', 'Z']]\
.astype({'X': 'int32', \
        'Y': 'int32', \
        'Z': 'int16'})\
.reset_index()

In [62]:
len(df_1m)

316682

In [63]:
## Deslocar X e Y de acordo com Azimute e inclinação

x, y = 100, 100

hag = 10 #Height Above Ground
z = 100
inclinacao = 45
azimute = 90

sombra = np.tan(inclinacao / 180. * np.pi) * z
print(sombra)

x = np.cos(azimute/180 * np.pi) * sombra
y = np.sin(azimute/180 * np.pi) * sombra

print(x, y)


99.99999999999999
6.123233995736765e-15 99.99999999999999


In [64]:
## Deslocar X e Y de acordo com theta e phi

x, y, z = 100, 100, 100

fusos = 60
meridianos = 30

thetas, delta_theta = np.linspace(-np.pi, np.pi, fusos, retstep=True, endpoint=False)
phis, delta_phi = np.linspace(1., 0., meridianos, retstep=True, endpoint=False)
np.arccos(phis)

theta = np.pi / 2
phi =  np.pi / 4

sombra = np.tan(phi) * z
print(sombra)


x = np.cos(theta) * sombra
y = np.sin(theta) * sombra

print(x, y)

99.99999999999999
6.123233995736765e-15 99.99999999999999


In [65]:
df_1m['X_proj'] = np.rint(df_1m.X + (np.cos(theta) * np.tan(np.arccos(phi)) * df_1m.Z)).astype('int32')
df_1m['Y_proj'] = np.rint(df_1m.Y + (np.sin(theta) * np.tan(np.arccos(phi)) * df_1m.Z)).astype('int32')

In [66]:
df_1m['sv'] = 0

In [67]:
XY_projZmax = df_1m[['X_proj', 'Y_proj', 'Z']].groupby(['X_proj', 'Y_proj'],  as_index=False).max()

In [68]:
XY_projZmax

,X_proj,Y_proj,Z
0,333058,7394943,-68
1,333058,7394945,-68
2,333058,7394946,-49
3,333058,7394947,-49
4,333058,7394948,-59
...,...,...,...
172327,333600,7395520,-58
172328,333600,7395521,-58
172329,333600,7395523,-58
172330,333600,7395524,-58


In [69]:
XY_projZmax['sv'] = 1
XY_projZmax

,X_proj,Y_proj,Z,sv
0,333058,7394943,-68,1
1,333058,7394945,-68,1
2,333058,7394946,-49,1
3,333058,7394947,-49,1
4,333058,7394948,-59,1
...,...,...,...,...
172327,333600,7395520,-58,1
172328,333600,7395521,-58,1
172329,333600,7395523,-58,1
172330,333600,7395524,-58,1


In [70]:
seila  = pd.merge(df_1m, XY_projZmax, on=['X_proj','Y_proj', 'Z'], how='left')['sv_y']

In [71]:
seila[seila.isna()] = 0

In [72]:
seila[seila==3] += np.nan

In [73]:
seila

0         0.0
1         1.0
2         1.0
3         1.0
4         1.0
         ... 
316677    0.0
316678    1.0
316679    0.0
316680    1.0
316681    1.0
Name: sv_y, Length: 316682, dtype: float64

## Iterando sobre $\theta$ e $\phi$

In [104]:
df_1m = \
df[(df.Classification == 2) | (df.Classification == 6)]\
                          [['X', 'Y', 'Z']]\
.astype({'X': 'int32', \
        'Y': 'int32', \
        'Z': 'int16'})\
.reset_index()

In [105]:
df_1m['sv'] = 0

In [110]:
for t in thetas:
# t = np.pi/2
    print(f'theta: {t}')
    for p in phis:
        if np.tan(np.arccos(p)) < 1:
            df_1m['X_proj'] = np.rint(df_1m.X + (np.cos(t) * np.tan(np.arccos(p)) * df_1m.Z)).astype('int32')
            df_1m['Y_proj'] = np.rint(df_1m.Y + (np.sin(t) * np.tan(np.arccos(p)) * df_1m.Z)).astype('int32')
        else:
            df_1m['X_proj'] = np.rint((df_1m.X + (np.cos(t) * np.tan(np.arccos(p)) * df_1m.Z)) / np.tan(np.arccos(p))).astype('int32')
            df_1m['Y_proj'] = np.rint((df_1m.Y + (np.sin(t) * np.tan(np.arccos(p)) * df_1m.Z)) / np.tan(np.arccos(p))).astype('int32')

    #     print(f'tg de arcocoseno: {np.tan(np.arccos(p))}')

        XY_projZmax = \
        df_1m[['X_proj', 'Y_proj', 'Z']].groupby(['X_proj', 'Y_proj'],  as_index=False).max()
        XY_projZmax['sv_proj'] = 1
    #     print(len(XY_projZmax))

        df_temp = \
        pd.merge(df_1m, XY_projZmax, on=['X_proj','Y_proj', 'Z'], how='left')['sv_proj']

        df_1m['sv'] = df_1m['sv'].fillna(0) + df_temp.fillna(0)  
#      df_1m_temp

theta: -3.141592653589793
theta: -3.036872898470133
theta: -2.9321531433504737
theta: -2.827433388230814
theta: -2.722713633111154
theta: -2.6179938779914944
theta: -2.5132741228718345
theta: -2.408554367752175
theta: -2.303834612632515
theta: -2.199114857512855
theta: -2.0943951023931957
theta: -1.9896753472735358
theta: -1.8849555921538759
theta: -1.7802358370342162
theta: -1.6755160819145565
theta: -1.5707963267948966
theta: -1.4660765716752369
theta: -1.3613568165555772
theta: -1.2566370614359172
theta: -1.1519173063162575
theta: -1.0471975511965979
theta: -0.9424777960769379
theta: -0.8377580409572785
theta: -0.7330382858376185
theta: -0.6283185307179586
theta: -0.5235987755982991
theta: -0.41887902047863923
theta: -0.3141592653589793
theta: -0.20943951023931984
theta: -0.10471975511965992
theta: 0.0
theta: 0.10471975511965947
theta: 0.2094395102393194
theta: 0.3141592653589793
theta: 0.4188790204786388
theta: 0.5235987755982987
theta: 0.6283185307179586
theta: 0.7330382858376181


In [111]:
1 - (df_1m.sv / (fusos * meridianos))

0         0.483333
1         0.448889
2         0.441111
3         0.417778
4         0.400000
            ...   
316677    0.623333
316678    0.483333
316679    0.462778
316680    0.427778
316681    0.476667
Name: sv, Length: 316682, dtype: float64

In [114]:
df_1m.sv.min()

6.0

In [113]:
df_1m.drop(['index', 'X_proj', 'Y_proj'], axis=1).to_csv('resultados/3314-231-pontos.csv')

In [103]:
phis

array([1.        , 0.96666667, 0.93333333, 0.9       , 0.86666667,
       0.83333333, 0.8       , 0.76666667, 0.73333333, 0.7       ,
       0.66666667, 0.63333333, 0.6       , 0.56666667, 0.53333333,
       0.5       , 0.46666667, 0.43333333, 0.4       , 0.36666667,
       0.33333333, 0.3       , 0.26666667, 0.23333333, 0.2       ,
       0.16666667, 0.13333333, 0.1       , 0.06666667, 0.03333333])

In [102]:
np.rint(99000 / np.tan(np.arccos(phis)))

<ipython-input-102-50eeeeb032ce>:1: RuntimeWarning: divide by zero encountered in true_divide
  np.rint(99000 / np.tan(np.arccos(phis)))


array([    inf, 373772., 257374., 204409., 171983., 149248., 132000.,
       118216., 106785.,  97039.,  88548.,  81020.,  74250.,  68087.,
        62418.,  57158.,  52237.,  47601.,  43207.,  39017.,  35002.,
        31134.,  27392.,  23756.,  20208.,  16734.,  13319.,   9950.,
         6615.,   3302.])

## Gerando raster com resultados